1)Сделайте Merge наборов данных (в случае с логами: "access_logs_csv", "official_browser_site_csv", "dict_site_status_code_csv").
2)Сделайте проверку пустых значений (в случае с логами по колонке merge)
3)Найдите список URL'ов, у которых есть посещением одним и тем же IP более 5 раз (т.е. один пользователь заходил 5 и более раз по одной и той же ссылке)
4)Сделайте сортировку и покажите 10 самых популярных
5)Найдите 10 самых посещаемых URL с 6 утра до 11 часов утра

In [127]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

In [6]:
spark = SparkSession.builder.appName("Basic").getOrCreate()

In [70]:
df_0 = pd.read_excel('perviy.xlsx')

In [71]:
df_1 = pd.read_excel('vtor.xlsx')

In [72]:
df_2 = pd.read_excel('tret.xlsx')

1)Сделайте Merge наборов данных (в случае с логами: "access_logs_csv", "official_browser_site_csv", "dict_site_status_code_csv").

In [83]:
df_3 = df_1.merge(df_0,  how='inner')

In [129]:
df_3['date_time'] = pd.to_datetime(df_3['date_time'])

In [130]:
df_3

,site_status_code,meaning,ip_adress,date_time,site,browser_raw
0,200,OK,209.160.24.63,2016-03-03 18:22:16,http://www.google.com,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...
1,200,OK,209.160.24.63,2016-03-03 18:22:16,http://www.exploratorystore.io/oldlink?itemId=...,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...
2,200,OK,209.160.24.63,2016-03-03 18:22:17,http://www.exploratorystore.io/product.screen?...,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...
3,200,OK,209.160.24.63,2016-03-03 18:22:19,http://www.exploratorystore.io/cart.do?action=...,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...
4,200,OK,209.160.24.63,2016-03-03 18:22:20,http://www.exploratorystore.io/category.screen...,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...
...,...,...,...,...,...,...
13623,505,NaN,233.77.49.94,2016-03-10 13:45:45,http://www.exploratorystore.io/category.screen...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8)...
13624,505,NaN,91.217.178.210,2016-03-10 16:42:01,http://www.exploratorystore.io/oldlink?itemId=...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_4)...
13625,505,NaN,91.217.178.210,2016-03-10 16:42:07,http://www.exploratorystore.io/product.screen?...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_4)...
13626,505,NaN,86.51.1.2,2016-03-10 16:57:39,http://www.exploratorystore.io/product.screen?...,Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US...


2)Сделайте проверку пустых значений (в случае с логами по колонке merge)

In [86]:
df_3.isnull().sum()

site_status_code      0
meaning             242
ip_adress             0
date_time             0
site                  0
browser_raw           0
dtype: int64

3)Найдите список URL'ов, у которых есть посещением одним и тем же IP более 5 раз (т.е. один пользователь заходил 5 и более раз по одной и той же ссылке)

In [109]:
gr = df_3.groupby(['site']).agg({'ip_adress' : ['count']})
gr

,ip_adress
,count
site,
http://www.bing.com,128
http://www.exploratorystore.io,640
http://www.exploratorystore.io/cart.do?action=addtocart&itemId=EST-11,3
http://www.exploratorystore.io/cart.do?action=addtocart&itemId=EST-11&categoryId=ACCESSORIES&productId=WC-SH-A01,5
http://www.exploratorystore.io/cart.do?action=addtocart&itemId=EST-11&categoryId=ACCESSORIES&productId=WC-SH-A02,6
...,...
http://www.exploratorystore.io/product.screen?productId=WC-SH-A02,197
http://www.exploratorystore.io/product.screen?productId=WC-SH-G04,259


In [112]:
list(gr)

[('ip_adress', 'count')]

In [115]:
result = gr[gr['ip_adress', 'count'] > 5]

In [116]:
result

,ip_adress
,count
site,
http://www.bing.com,128
http://www.exploratorystore.io,640
http://www.exploratorystore.io/cart.do?action=addtocart&itemId=EST-11&categoryId=ACCESSORIES&productId=WC-SH-A02,6
http://www.exploratorystore.io/cart.do?action=addtocart&itemId=EST-11&categoryId=ARCADE&productId=FI-AG-G08,8
http://www.exploratorystore.io/cart.do?action=addtocart&itemId=EST-11&categoryId=ARCADE&productId=MB-AG-G07,6
...,...
http://www.exploratorystore.io/product.screen?productId=WC-SH-A02,197
http://www.exploratorystore.io/product.screen?productId=WC-SH-G04,259


4)Сделайте сортировку и покажите 10 самых популярных

In [123]:
res = result.sort_values(by=('ip_adress', 'count'), ascending=False)

In [125]:
res.head(10)

,ip_adress
,count
site,
http://www.exploratorystore.io/category.screen?categoryId=STRATEGY,799
http://www.exploratorystore.io,640
http://www.google.com,546
http://www.exploratorystore.io/category.screen?categoryId=ARCADE,473
http://www.exploratorystore.io/category.screen?categoryId=NULL,434
http://www.exploratorystore.io/product.screen?productId=SF-BVS-G01,426
http://www.exploratorystore.io/category.screen?categoryId=ACCESSORIES,383
http://www.exploratorystore.io/category.screen?categoryId=TEE,344


5)Найдите 10 самых посещаемых URL с 6 утра до 11 часов утра

In [141]:
resultf = df_3[df_3['date_time'].dt.hour.between(6, 11)].groupby(['site']).agg({'ip_adress' : ['count']})

In [143]:
resultf = resultf[resultf['ip_adress', 'count'] > 5]

In [144]:
resultf = resultf.sort_values(by=('ip_adress', 'count'), ascending=False)

In [147]:
resultf.head(10)

,ip_adress
,count
site,
http://www.exploratorystore.io/category.screen?categoryId=STRATEGY,171
http://www.exploratorystore.io,134
http://www.google.com,117
http://www.exploratorystore.io/category.screen?categoryId=NULL,90
http://www.exploratorystore.io/product.screen?productId=SF-BVS-G01,88
http://www.exploratorystore.io/category.screen?categoryId=ACCESSORIES,87
http://www.exploratorystore.io/category.screen?categoryId=ARCADE,81
http://www.exploratorystore.io/category.screen?categoryId=TEE,72
